In [0]:
# Counting and aggregating M&M's (Python version)


# Import the necessary libraries
# Since we are using Python, import the SparkSession and related funcionts from PySpark module.

import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import count

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: mnmcount <file>", file=sys.stderr)
        sys.exit(-1)


Usage: mnmcount <file>
/databricks/python/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


An exception has occurred, use %tb to see the full traceback.

SystemExit: -1


In [0]:
# Build a SparkSession using the SparkSession APIs.
# If one does not exist, then create an instance. There can only be one SparkSession per JVM.

spark = (SparkSession.builder.appName("PythonMnMCount").getOrCreate())


In [0]:
# Get the M&M data set filename from the command-line arguments
mnm_file=sys.argv[1]


In [0]:
#Read the file into a Spark DF using the CSV format by inferring the schema and specifying that the file contains a header, which provides
# column names for comma-separated fields.

mnm_df = (spark.read.format("csv").option("header","true").option("inferSchema","true").csv("/FileStore/shared_uploads/irene.castro@bosonit.com/mnm_dataset.csv"))

In [0]:
# We use the DF high-level APIs. Note that we don't use RDDs at all. Because some of Spark's functions return the same object, we can chain 
# function calls.
# 1. Select from the DF the fiels "State", "Color" and "Count".
# 2. Since we want to group each state and its M&M color count, we use groupBy()
# 3. Aggregate counts of all colors and groupBy() State and Color.
# 4. orderBy() in descending order

count_mnm_df=(mnm_df.select("State","Color","Count").groupBy("State","Color").agg(count("Count").alias("Total")).orderBy("Total", ascending=False))

In [0]:
# Show the resulting aggregations for all the states and colors; a total count of each color per state.
# Note show() is an action, which will trigger the above query to be executed.

count_mnm_df.show(n=60,truncate=False)
print("Total Rows = %d" % (count_mnm_df.count()))

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|WA   |Green |1779 |
|OR   |Orange|1743 |
|TX   |Green |1737 |
|TX   |Red   |1725 |
|CA   |Green |1723 |
|CO   |Yellow|1721 |
|CA   |Brown |1718 |
|CO   |Green |1713 |
|NV   |Orange|1712 |
|TX   |Yellow|1703 |
|NV   |Green |1698 |
|AZ   |Brown |1698 |
|WY   |Green |1695 |
|CO   |Blue  |1695 |
|NM   |Red   |1690 |
|AZ   |Orange|1689 |
|NM   |Yellow|1688 |
|NM   |Brown |1687 |
|UT   |Orange|1684 |
|NM   |Green |1682 |
|UT   |Red   |1680 |
|AZ   |Green |1676 |
|NV   |Yellow|1675 |
|NV   |Blue  |1673 |
|WA   |Red   |1671 |
|WY   |Red   |1670 |
|WA   |Brown |1669 |
|NM   |Orange|1665 |
|WY   |Blue  |1664 |
|WA   |Yellow|1663 |
|WA   |Orange|1658 |
|CA   |Orange|1657 |
|NV   |Brown |1657 |
|CA   |Red   |1656 |
|CO   |Brown |1656 |
|UT   |Blue  |1655 |
|AZ   |Yellow|1654 |
|TX   |Orange|1652 |
|AZ   |Red   |1648 |
|OR   |Blue  |1646 |
|UT   |Yellow|1645 |
|OR   |Red   |1645 |
|CO   |Orange|1642 |
|TX   |Brown 

In [0]:
# While the above code aggregated and counter for all the states, what if we just want to see the data for a single state, e.g., CA?
# 1. Select from all rows in the DF
# 2. Filter only CA state
# 3. groupBy() State and Color as we did above.
# 4. Aggregate the counts for each color
# 5. orderBy() in descending order
# Find the aggregate count for California by filtering.

ca_count_mnm_df=(mnm_df.select("State","Color","Count").where(mnm_df.State=="CA").groupBy("State","Color").agg(count("Count").alias("Total")).orderBy("Total",ascending=False))

In [0]:
# Show the resulting aggregation for California. As above, show() is an action that will trigger the execution of the entire computation.

ca_count_mnm_df.show(n=10, truncate=False)

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|CA   |Green |1723 |
|CA   |Brown |1718 |
|CA   |Orange|1657 |
|CA   |Red   |1656 |
|CA   |Blue  |1603 |
+-----+------+-----+



In [0]:
# Stop the SparkSession
spark.stop()